In [2]:
from scipy.io import loadmat
import torch
import numpy as np
from tqdm import tqdm

Gchannel = torch.load('Gchannel.pt')
print(Gchannel.shape)
########### pilot signals ############
num_pilots = 100
Nt = 64

# 创建包含给定复数的实部和虚部的张量
constellation_real = torch.tensor([1, 1, -1, -1], dtype=torch.float)
constellation_imag = torch.tensor([1, -1, 1, -1], dtype=torch.float)

# 生成随机索引
p_indices = torch.randint(0, len(constellation_real), (Nt,))
# 使用索引分别从实部和虚部张量中选择元素
random_real_parts = constellation_real[p_indices]
random_imag_parts = constellation_imag[p_indices]

# 将实部和虚部合并成一个复数张量
pilot = torch.complex(random_real_parts, random_imag_parts)
print(pilot.shape)

########### Pick Validation index ##############
val_number = 10
val_index = torch.randint(0, len(Gchannel), (val_number,))
val_channel = torch.zeros([Nt,Nt,2,val_number])
for i in range(len(val_index)):
    val_channel[:,:,:,i] = Gchannel[:,:,:,val_index[i]]
val_channel = torch.complex(torch.FloatTensor(val_channel[:,:,0,:]), torch.FloatTensor(val_channel[:,:,1,:]))
print(val_channel.shape)
HP = torch.zeros_like(val_channel)
for i in range(val_channel.shape[2]):
    HP[:,:,i] = torch.matmul(val_channel[:,:,i], pilot)

########## AWGN noise ##############
noise = torch.rand_like(HP)+1j*torch.rand_like(HP)
snr_range = torch.arange(-10, 30, 5)
noise_power = 10 ** (-snr_range / 10)

C:\Users\Zhizhou He\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([64, 64, 2, 96])
torch.Size([64])
torch.Size([64, 64, 10])


In [8]:
T = 300 ### total time slots in forward or backward process
steps = 3 ########## iteration steps at each noise level
random_channel = torch.randn_like(val_channel)

################## For each SNR ###################
for snr_idx, local_noise in tqdm(enumerate(noise_power)):
    val_received = HP + torch.sqrt(noise_power[snr_idx])*noise # received signal through real channel
    channel_t = random_channel.clone()
    testing_index = 0
    val_P_hermitian = torch.conj(torch.transpose(pilot, -1, 0))
    ############### For each time slot in backward process #############
    for time_idx in tqdm(range(T)):
        labels = torch.ones(random_channel.shape[2]) * time_idx
        ######### For each step at that noise level ########
        for step_idx in range(steps):
            with torch.no_grad():
                score = model(channel_t, labels)
                # Compute gradient for measurements in un-normalized space
                meas_grad = torch.matmul(val_P_hermitian, torch.matmul(pilot, channel_t) - val_recevied)
                # Sample noise
                grad_noise = np.sqrt(2 * alpha * beta_noise) * torch.randn_like(channel_t)
                # Apply update
                channel_t = channel_t + alpha * (score - meas_grad /(local_noise/2. + current_sigma ** 2)) + grad_noise
                
                # Store loss
                nmse_log[snr_idx, testing_idx] = \
                (torch.sum(torch.square(torch.abs(current - oracle)), dim=(-1, -2))/\
                torch.sum(torch.square(torch.abs(oracle)), dim=(-1, -2))).cpu().numpy()
                testing_idx = testing_idx + 1

ModuleNotFoundError: No module named 'train_score'

In [29]:
print(snr_idx)
print((val_channel.shape[2]))
print(random_channel.shape)

0
10
torch.Size([64, 64, 10])
